In [20]:
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.2.23479.6"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0-rc.2.23479.6"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0-rc.2.23479.6"
#r "nuget: Microsoft.Extensions.Logging.Debug, 8.0.0-rc.2.23479.6"
#r "nuget: Microsoft.OpenApi, 1.6.10"
#r "nuget: Microsoft.SemanticKernel.Core, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.OpenAI, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Postgres, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Functions.Grpc, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Functions.OpenAPI, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Plugins.Document, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Planners.Core, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta7"
#r "nuget: Microsoft.SemanticKernel.TemplateEngine.Basic, 1.0.0-beta7"
#r "nuget: System.CommandLine,2.0.0-beta4.22272.1"
#r "nuget: YamlDotNet,13.5.2"

Installed Packages Microsoft.Extensions.Configuration.Json, 8.0.0-rc.2.23479.6 Microsoft.Extensions.Configuration.UserSecrets, 8.0.0-rc.2.23479.6 Microsoft.Extensions.Logging.Console, 8.0.0-rc.2.23479.6 Microsoft.Extensions.Logging.Debug, 8.0.0-rc.2.23479.6 Microsoft.OpenApi, 1.6.10 microsoft.semantickernel.abstractions, 1.0.0-beta7 Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta7 microsoft.semantickernel.connectors.ai.openai, 1.0.0-beta7 Microsoft.SemanticKernel.Connectors.Memory.Postgres, 1.0.0-beta7 Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 1.0.0-beta7 Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 1.0.0-beta7 microsoft.semantickernel.core, 1.0.0-beta7 Microsoft.SemanticKernel.Functions.Grpc, 1.0.0-beta7 Microsoft.SemanticKernel.Functions.OpenAPI, 1.0.0-beta7 microsoft.semantickernel.planners.core, 1.0.0-beta7 microsoft.semantickernel.plugins.core, 1.0.0-beta7 Microsoft.SemanticKernel.Plugins.Document, 1.0.0-beta7 microsoft.semantickernel.plugins.memory, 1.0.0-beta7 Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta7 microsoft.semantickernel.templateengine.basic, 1.0.0-beta7 System.CommandLine, 2.0.0-beta4.22272.1 YamlDotNet, 13.5.2

In [21]:
using System;
using System.IO;
using System.CommandLine;
using System.Collections.Immutable;
using System.Text.RegularExpressions;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;
using Microsoft.SemanticKernel.Functions.OpenAPI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.TemplateEngine.Basic;

In [22]:
var notebookDir = Environment.CurrentDirectory;
var parentDir = Directory.GetParent(notebookDir).FullName;

var builder = new ConfigurationBuilder()
    .SetBasePath(parentDir)  // Set the base path
    .AddJsonFile("config/appsettings.json", optional: false, reloadOnChange: true)  // Load the main appsettings.json file
    .AddJsonFile("appsettings.plugins.json", optional: true, reloadOnChange: true);  // Load the environment-specific appsettings file

var conf = builder.Build();

In [23]:
ILoggerFactory loggerFactory = LoggerFactory.Create(builder =>
{
  builder.SetMinimumLevel(LogLevel.Warning)
          .AddConsole()
          .AddDebug();
});

In [26]:
var serviceId = conf.GetValue<string>("serviceId");
var endpointType = conf.GetValue<string>("endpointType");
var serviceType = conf.GetValue<string>("serviceType");
var deploymentOrModelId = conf.GetValue<string>("deploymentOrModelId");
var apiKey = conf.GetValue<string>("apiKey");
var orgId = conf.GetValue<string>("orgId");

// Taken from: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example42_KernelBuilder.cs
//
// KernelBuilder returns a new builder instance, in case you want to configure the builder differently.
// ==========================================================================================================
// Kernel instances can be created the usual way with "new", though the process requires particular
// attention to how dependencies are wired together. Although the building blocks are available
// to enable custom configurations, we highly recommend using KernelBuilder instead, to ensure
// a correct dependency injection.
// ----------------------------------------------------------------------------------------------------------
IKernel kernel = new KernelBuilder()
                     //.WithLoggerFactory(loggerFactory)
                     .WithOpenAITextCompletionService(
                            modelId: deploymentOrModelId,
                            apiKey: apiKey,
                            orgId: orgId,
                            serviceId: serviceId)
                    .Build();

In [27]:
var rootDirectory = conf.GetSection("PluginSettings:Root").Get<string>();
var pluginDirectories = conf.GetSection("PluginSettings:Plugins").Get<string[]>();

var pluginsRoot = Path.Combine(parentDir, rootDirectory!);
var pluginImport = kernel.ImportSemanticFunctionsFromDirectory(pluginsRoot, pluginDirectories!);

In [28]:
int nameMaxLength = pluginImport.Max(p => p.Value.Name.Length) + 2;  // +2 for padding
int descMaxLength = pluginImport.Max(p => p.Value.Description.Length) + 2;

// Header
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");
Console.WriteLine($"|{"Plugin".PadRight(nameMaxLength)}|{"Description".PadRight(descMaxLength)}|");
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");

// Rows
foreach (var p in pluginImport)
{
    Console.WriteLine($"|{p.Value.Name.PadRight(nameMaxLength)}|{p.Value.Description.PadRight(descMaxLength)}|");
}

// Footer
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");


+---------------+----------------------------------------------------+
|Plugin         |Description                                         |
+---------------+----------------------------------------------------+
|Chat           |Answer user questions                               |
|Kubernetes     |Create kubernetes YAML files for given devops task  |
|Helm           |Create Helm Charts for given devops task            |
|TypeScript     |Create TypeScript source codes for given task       |
|CSharp         |Create .NET projects for given task                 |
|CreateHtmlDoc  |Create HTML documents                               |
|ExtractJS      |Extract JavaScript from an HTML document            |
|UrlFinder      |Find the URL of a web page                          |
+---------------+----------------------------------------------------+


In [29]:
string description = "create a kubernetes secret YAML for user/password";
var context = new ContextVariables();

string key = "input";
context.Set(key, description);

var result = await kernel.RunAsync(context, pluginImport["Kubernetes"]);
Console.WriteLine(result.GetValue<string>());


FILE: secret.yaml
#----#
apiVersion: v1
kind: Secret
metadata:
  name: user-password
type: Opaque
data:
  username: dXNlcg==
  password: cGFzc3dvcmQ=
#----#
